<a href="https://colab.research.google.com/github/blessondensil294/AV-Computer-Vision-Classification/blob/master/JanataHack_Computer_Vision_Hackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# JanataHack: Computer Vision Hackathon

Computer Vision as a field of research is notoriously difficult. Almost no research problem has been satisfactorily solved. One main reason for this difficulty is that the human visual system is simply too good for many tasks (e.g., face recognition), so that computer vision systems suffer by comparison. 

A human can recognize faces under all kinds of variations in illumination, viewpoint, expression, etc. In most cases we have no difficulty in recognizing a friend in a photograph taken many years ago. Also, there appears to be no limit on how many faces we can store in our brains for future recognition. There appears no hope in building an autonomous system with such stellar performance.

Given all this, computer vision has shown a lot of promise and reshaping the future of various industries such as automobile industry, healthcare industry, financial industry and so on

This weekend we bring to you another hackathon to apply your deep learning skills to solve a computer vision problem. This time you are in for a two week ride so brush up on our computer vision skills and hop on.

**Emergency vs Non-Emergency Vehicle Classification**

Fatalities due to traffic delays of emergency vehicles such as ambulance & fire brigade is a huge problem. In daily life, we often see that emergency vehicles face difficulty in passing through traffic. So differentiating a vehicle into an emergency and non emergency category can be an important component in traffic monitoring as well as self drive car systems as reaching on time to their destination is critical for these services.

In this problem, you will be working on classifying vehicle images as either belonging to the emergency vehicle or non-emergency vehicle category. For the same, you are provided with the train and the test dataset. Emergency vehicles usually includes police cars, ambulance and fire brigades.

# Load the Data from Google

## Loading from PyDrive

In [0]:
!pip install PyDrive

In [0]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

import pandas as pd
import numpy as np

from zipfile import ZipFile
from PIL import Image

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
download = drive.CreateFile({'id': '1ysraoYgWog0P07xqMcXsZfOtvqZ6ZBB3'})
download.GetContentFile('train.csv')
df_Train = pd.read_csv('train.csv')

download = drive.CreateFile({'id': '1iMxhd9ONDy8bnnf7_I_1nVFZ0YDuXL4q'})
download.GetContentFile('test_vc2kHdQ.csv')
df_Test = pd.read_csv('test_vc2kHdQ.csv')

download = drive.CreateFile({'id': '1o2KCMX32H1bEB0VG5s1S3uFy_BSrQMPQ'})
download.GetContentFile('sample_submission_yxjOnvz.csv')
df_sample_submission = pd.read_csv('sample_submission_yxjOnvz.csv')

download = drive.CreateFile({'id': '1KXOFMANqofzl6uQr-hPN7iFyHgs5n0rZ'})
download.GetContentFile('train_SOaYf6m.zip')
!unzip train_SOaYf6m.zip -d "cv_dataset"

Archive:  train_SOaYf6m.zip
replace cv_dataset/images/0.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: cv_dataset/images/0.jpg  
  inflating: cv_dataset/images/1.jpg  
  inflating: cv_dataset/images/10.jpg  
  inflating: cv_dataset/images/100.jpg  
  inflating: cv_dataset/images/1000.jpg  
  inflating: cv_dataset/images/1001.jpg  
  inflating: cv_dataset/images/1002.jpg  
  inflating: cv_dataset/images/1003.jpg  
  inflating: cv_dataset/images/1004.jpg  
  inflating: cv_dataset/images/1005.jpg  
  inflating: cv_dataset/images/1006.jpg  
  inflating: cv_dataset/images/1007.jpg  
  inflating: cv_dataset/images/1008.jpg  
  inflating: cv_dataset/images/1009.jpg  
  inflating: cv_dataset/images/101.jpg  
  inflating: cv_dataset/images/1010.jpg  
  inflating: cv_dataset/images/1011.jpg  
  inflating: cv_dataset/images/1012.jpg  
  inflating: cv_dataset/images/1013.jpg  
  inflating: cv_dataset/images/1014.jpg  
  inflating: cv_dataset/images/1015.jpg  
  inflating: cv_dataset/ima

## load using Mount

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import numpy as np
train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/test_vc2kHdQ.csv')

In [0]:
pwd

'/content'

In [0]:
ls -ltr

total 8
drwxr-xr-x 1 root root 4096 May 29 18:19 sample_data/
drwx------ 4 root root 4096 Jun  6 05:46 drive/


In [0]:
download.GetContentFile('/content/drive/My Drive/Colab Notebooks/train_SOaYf6m.zip')
!unzip train_LbELtWX.zip

NameError: ignored

# Image Folder Creation

In [0]:
directory_train_emer='train/emergency'
directory_train_non_emer='train/not_emergency'

directory_vaild_emer='valid/emergency'
directory_valid_non_emer='valid/not_emergency'

parent_directory='cv'

path1=os.path.join(parent_directory, directory_train_emer)
os.makedirs(path1, exist_ok=True)
path1=os.path.join(parent_directory, directory_train_non_emer)
os.makedirs(path1, exist_ok=True)
path1=os.path.join(parent_directory, directory_vaild_emer)
os.makedirs(path1, exist_ok=True)
path1=os.path.join(parent_directory, directory_valid_non_emer)
os.makedirs(path1, exist_ok=True)

# Split the Data to Train and Test

In [0]:
#File_Split
dataset_train=pd.read_csv('cv_dataset/train.csv')

path_directory='cv_dataset/images'
for x in dataset_train.query('emergency_or_not==1')['image_names']:
    path2=os.path.join(path_directory, x)
    img=Image.open(path2)
    path_emer=os.path.join(parent_directory, directory_train_emer)
    path_emer=os.path.join(path_emer,x)
    img.save(path_emer)

for x in dataset_train.query('emergency_or_not==0')['image_names']:
    path2=os.path.join(path_directory, x)
    img=Image.open(path2)
    path_emer=os.path.join(parent_directory, directory_train_non_emer)
    path_emer=os.path.join(path_emer,x)
    img.save(path_emer)

# Model using Convolution CNN

In [10]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten

from keras.layers import Dense, Dropout
from keras.layers import BatchNormalization

Using TensorFlow backend.


In [11]:
classifier= Sequential()

classifier.add(Conv2D(32,(3,3), padding='same',  input_shape=(224,224,3), activation = "relu"))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size=(2,2))) 

classifier.add(Conv2D(64,(3,3), padding='same', activation = "relu"))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size=(2,2)))
classifier.add(Flatten())

classifier.add(Dense(64, activation = 'relu', kernel_initializer='uniform'))
classifier.add(Dropout(0.2))

classifier.add(Dense(64, activation = 'relu', kernel_initializer='uniform'))
classifier.add(Dropout(0.2))

classifier.add(Dense(1, activation = 'sigmoid', kernel_initializer='uniform'))

classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)

train_set = train_datagen.flow_from_directory('cv/train', 
                                              target_size=(224,224), 
                                              batch_size=32, class_mode='binary')

# test_set = test_datagen.flow_from_directory('drive/My Drive/classification/dataset/test_set', 
#                                             target_size=(180,180), 
#                                             batch_size=32, class_mode='binary')

#filepath='classification_main.hdf5'
#best_model_save=ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
# monitor=EarlyStopping(monitor='val_accuracy', patience=5, verbose=1)
classifier.fit_generator(train_set, epochs=50)

Found 1646 images belonging to 2 classes.
Epoch 1/50
52/52 [==============================] - 186s 4s/step - loss: 0.6460 - accuracy: 0.6817
Epoch 2/50
52/52 [==============================] - 181s 3s/step - loss: 0.5104 - accuracy: 0.7546
Epoch 3/50
52/52 [==============================] - 180s 3s/step - loss: 0.4419 - accuracy: 0.8013
Epoch 4/50
52/52 [==============================] - 184s 4s/step - loss: 0.4047 - accuracy: 0.8129
Epoch 5/50
52/52 [==============================] - 184s 4s/step - loss: 0.3794 - accuracy: 0.8262
Epoch 6/50
52/52 [==============================] - 184s 4s/step - loss: 0.3671 - accuracy: 0.8335
Epoch 7/50
52/52 [==============================] - 187s 4s/step - loss: 0.3366 - accuracy: 0.8645
Epoch 8/50
52/52 [==============================] - 183s 4s/step - loss: 0.2939 - accuracy: 0.8736
Epoch 9/50
52/52 [==============================] - 183s 4s/step - loss: 0.3015 - accuracy: 0.8767
Epoch 10/50
52/52 [==============================] - 182s 4s/step -

KeyboardInterrupt: ignored

# Model using VGG16

In [0]:
from keras import applications
from keras.utils.np_utils import to_categorical
import math

img_width, img_height = 224,224
top_model_weights_path = 'classification_model.h5'
epochs = 50
batch_size = 64

#Importing Vgg16 without Top layer 
vgg16 = applications.VGG16(include_top=False, weights='imagenet')
datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True) 

train_set = datagen.flow_from_directory( 
       'cv/train', 
        target_size=(img_width, img_height), 
        batch_size=batch_size, 
        class_mode=None, 
        shuffle=False) 
     
nb_train_samples = len(train_set.filenames) 
num_classes = len(train_set.class_indices)  
predict_size_train = int(math.ceil(nb_train_samples / batch_size))
     
train_features = vgg16.predict_generator(train_set, predict_size_train)
np.save('train_features.npy', train_features)

train_set_lables = datagen.flow_from_directory( 
   'cv/train', 
   target_size=(img_width, img_height), 
   batch_size=batch_size, 
   class_mode='binary', 
   shuffle=False) 
 
nb_train_samples = len(train_set_lables.filenames) 
num_classes = len(train_set_lables.class_indices)  
train_data = np.load('train_features.npy')  
train_labels = train_set_lables.classes  
train_labels = to_categorical(train_labels, num_classes=num_classes)

#Building own Top Layer
classifier = Sequential() 
classifier.add(Flatten(input_shape=train_data.shape[1:])) 

classifier.add(Dense(32, activation = 'relu', kernel_initializer='he_uniform')) # he_uniform
classifier.add(Dropout(0.1))

classifier.add(Dense(32, activation = 'relu', kernel_initializer='he_uniform')) # he_uniform
classifier.add(Dropout(0.1))

# classifier.add(Dense(64, activation = 'relu', kernel_initializer='he_uniform')) # he_uniform
# classifier.add(Dropout(0.1))


classifier.add(Dense(num_classes, activation = 'softmax', kernel_initializer='he_uniform')) # he_uniform
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
classifier.fit(train_data, train_labels, epochs=10, batch_size=batch_size)

# Prediction of Test Images

In [0]:
#Predicting Test Images
# dataset_test=pd.read_csv('drive/My Drive/test_vc2kHdQ.csv')
# dataset_submission=pd.read_csv('drive/My Drive/Analytics_Vidya_cv/sample_submission_yxjOnvz.csv')
path_directory='cv_dataset/images'
dataset_test=test
dataset_submission= sample_submission
main_result=[]
for x in dataset_test['image_names']:
    path2=os.path.join(path_directory, x)
    test_image=image.load_img(path2, target_size=(img_height,img_width))
    test_image=image.img_to_array(test_image)
    test_image=np.expand_dims(test_image, axis=0)
    bt_prediction = vgg16.predict(test_image)
    result=classifier.predict(bt_prediction)
    main_result.append(result[0][0])

dataset_submission['emergency_or_not']=main_result 
dataset_submission['emergency_or_not']=np.where(dataset_submission['emergency_or_not']<1, 0, 1)
dataset_submission.head(20)

In [0]:
file_name = "Sample_CNN_45_VGG_adam_10_Epochs_he_uniform_Extra_Dense_32_32_Dropout_0pt1"
dataset_submission.to_csv(file_name+'.csv',header=True, index=False)

from google.colab import files
files.download(file_name+'.csv')